In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras import Sequential, layers, Input, regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

2025-06-11 11:27:28.676005: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-11 11:27:28.676697: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 11:27:28.680250: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 11:27:28.688572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749634048.703378   36713 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749634048.70

In [2]:
df = pd.read_csv('../data/data_clean.csv')
df.head(5)

/tmp/ipykernel_36713/2597316256.py:1: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/data_clean.csv')


,uid,id,nature,acheteur_id,acheteur_nom,acheteur_siren,titulaire_id,titulaire_typeIdentifiant,titulaire_nom,titulaire_siren,...,codeCPV_2,codeCPV_3,codeCPV_4,codeCPV_5,codeCPV_2_3,annee,acheteur_tranche_effectif,acheteur_categorie,titulaire_tranche_effectif,titulaire_categorie
0,210601209000132022_M013,2022_M013,Marché,21060120900013,COMMUNE DE SAINT ETIENNE DE TINEE,210601209.0,38177692100029,SIRET,SERMATECH,381776921.0,...,45000000,45200000,45210000,45210000,45200000,2021,21,PME,12,PME
1,217100759000182024RENOCHARP,2024RENOCHARP,Marché,21710075900018,COMMUNE DE CHALMOUX,217100759.0,75203574100038,SIRET,MACON ETANCHEITE,752035741.0,...,45000000,45200000,45260000,45261000,45200000,2023,11,PME,12,PME
2,200066231000162022033INFOL00,2022033INFOL00,Marché,20006623100016,CC DES PORTES D'ARIEGE PYRENEES,200066231.0,49459697600014,SIRET,EQUADEX,494596976.0,...,72000000,72500000,72510000,72514000,72000000,2023,31,PME,21,PME
3,243100518001702024M05,2024M05,Marché,24310051800170,TOULOUSE METROPOLE,243100518.0,59278023300017,SIRET,RIVES & EAUX DU SUD-OUEST,592780233.0,...,71000000,71300000,71300000,71300000,71300000,2024,52,GE,31,PME
4,21590544900017202402401,202402401,Marché,21590544900017,COMMUNE DE SAINT SAULVE,215905449.0,32683156700010,SIRET,ALTOMARE ALTALU,326831567.0,...,44000000,44300000,44310000,44316000,44000000,2024,32,ETI,12,PME


In [3]:
from scripts.data_cleaner import filter_top_cpv_categories

df = filter_top_cpv_categories(df, top_n=150, cpv_column='codeCPV_3')

Filtered from 392 to 150 CPV categories, keeping 279174 rows out of 286850


In [4]:
df['codeCPV_3'].value_counts()

codeCPV_3
45200000    62229
45400000    19048
71200000    17621
45100000    13647
71300000    13462
            ...  
31300000      139
55100000      132
31100000      130
18800000      130
16100000      128
Name: count, Length: 150, dtype: int64

In [5]:
df.drop(df[df['montant'] > 999999].index, inplace=True)
df.shape

(238163, 42)

In [6]:
y = np.log1p(df['montant'])
y.head()

0    11.950698
1    11.460800
2    12.765691
4    10.615726
5    12.065219
Name: montant, dtype: float64

In [7]:
bins = np.linspace(y.min(), y.max(), 10)
bins = np.delete(bins, 1)

#bins = bins[2:]

# Créer les étiquettes correspondantes
labels = list(range(0, len(bins)-1))

# Ajouter la colonne fourchette_de_prix
df['bins'] = pd.cut(y, bins=bins, labels=labels, include_lowest=True, right=True)

df.shape

(238163, 43)

In [8]:
np.exp(bins)

array([  20001.        ,   47708.50133762,   73683.08597334,
        113799.36502582,  175756.69245133,  271446.28561345,
        419233.45817256,  647482.39989405, 1000000.        ])

In [9]:
len(labels)

8

In [10]:
df['bins'] = df['bins'].astype('int')
df = df.drop(columns='montant')
df.columns

Index(['uid', 'id', 'nature', 'acheteur_id', 'acheteur_nom', 'acheteur_siren',
       'titulaire_id', 'titulaire_typeIdentifiant', 'titulaire_nom',
       'titulaire_siren', 'objet', 'codeCPV', 'procedure', 'dureeMois',
       'dateNotification', 'datePublicationDonnees', 'formePrix',
       'attributionAvance', 'offresRecues', 'marcheInnovant', 'ccag',
       'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'tauxAvance',
       'origineUE', 'origineFrance', 'lieuExecution_code',
       'lieuExecution_typeCode', 'idAccordCadre', 'source_open_data',
       'codeCPV_FR', 'codeCPV_2', 'codeCPV_3', 'codeCPV_4', 'codeCPV_5',
       'codeCPV_2_3', 'annee', 'acheteur_tranche_effectif',
       'acheteur_categorie', 'titulaire_tranche_effectif',
       'titulaire_categorie', 'bins'],
      dtype='object')

In [11]:
X = df.drop(columns=['bins', 'titulaire_tranche_effectif', 'titulaire_categorie'])
y = df['bins']

In [12]:
X[['acheteur_tranche_effectif', 'acheteur_categorie']] = X[['acheteur_tranche_effectif', 'acheteur_categorie']].fillna('null')
X[['acheteur_tranche_effectif', 'acheteur_categorie']].isna().sum()

acheteur_tranche_effectif    0
acheteur_categorie           0
dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=X['codeCPV_3'])

In [14]:
from scripts.preprocess_pipeline import create_pipeline

numerical_columns = ['dureeMois', 'offresRecues', 'annee']

binary_columns = ['sousTraitanceDeclaree', 'origineFrance',
                          'marcheInnovant', 'idAccordCadre']

categorical_columns = ['procedure', 'nature', 'formePrix', 'ccag',
                               'typeGroupementOperateurs', 'tauxAvance_cat',
                               'codeCPV_3', 'acheteur_tranche_effectif', 'acheteur_categorie']

pipeline = create_pipeline(numerical_columns, binary_columns, categorical_columns)

In [15]:
X_train_preproc = pipeline.fit_transform(X_train)
X_train_preproc.shape

(190530, 220)

In [16]:
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_train_cat.shape

(190530, 8)

In [17]:
y_train_cat.shape

(190530, 8)

In [18]:
def build_mlp_model(input_dim=220, num_classes=8, use_dropout=True, l2_factor=1e-4):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))

    # Couche 1
    model.add(layers.Dense(
        1028, activation='relu',
        kernel_regularizer=regularizers.l2(l2_factor)
    ))
    model.add(layers.BatchNormalization())
    if use_dropout:
        model.add(layers.Dropout(0.4))

    # Couche 2
    model.add(layers.Dense(
        512, activation='relu',
        kernel_regularizer=regularizers.l2(l2_factor)
    ))
    model.add(layers.BatchNormalization())
    if use_dropout:
        model.add(layers.Dropout(0.3))

    # Couche 3
    model.add(layers.Dense(
        256, activation='relu',
        kernel_regularizer=regularizers.l2(l2_factor)
    ))
    model.add(layers.BatchNormalization())
    if use_dropout:
        model.add(layers.Dropout(0.2))

    # Sortie
    model.add(layers.Dense(
        num_classes, activation='softmax',
        kernel_regularizer=regularizers.l2(l2_factor)
    ))

    # Compilation
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [19]:
model = build_mlp_model()
model.summary()

2025-06-11 11:27:36.441898: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1028)           │       227,188 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1028)           │         4,112 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1028)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       526,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 894,604 (3.41 MB)

 Trainable params: 891,012 (3.40 MB)

 Non-trainable params: 3,592 (14.03 KB)

In [ ]:
es = EarlyStopping(patience=8, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=5, verbose=1, min_lr=1e-6)


history = model.fit(
        X_train_preproc, y_train_cat,
        validation_split=0.2,
        batch_size=128,
        epochs=150,
        callbacks=[es, lr_scheduler],
        verbose=1)

Epoch 1/150
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - accuracy: 0.1857 - loss: 2.3028 - val_accuracy: 0.2160 - val_loss: 2.0248 - learning_rate: 0.0010
Epoch 2/150
   6/1191 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.2122 - loss: 2.0123

In [ ]:
# Récupération des données d'entraînement et validation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Création de la figure
plt.figure(figsize=(12, 5))

# Plot de l'accuracy
plt.subplot(1, 2, 1)
plt.plot(acc, label='Accuracy')
plt.plot(val_acc, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plot de la loss
plt.subplot(1, 2, 2)
plt.plot(loss, label='Loss')
plt.plot(val_loss, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Affichage
plt.tight_layout()
plt.show()

In [243]:
model.save("modele_montant_8.keras")

In [ ]:
X_test_preproc = pipeline.transform(X_test)

In [245]:
y_test_cat = to_categorical(y_test)

In [ ]:
print(X_test_preproc.shape)
print(y_test_cat.shape)

In [247]:
# from keras.models import load_model

# modelf = load_model("model_montant.keras")

In [ ]:
y_new = model.predict(X_test_preproc)

In [ ]:
y_new.shape

In [ ]:
y_new

In [ ]:
# y_new : array de shape (n_samples, n_classes)
one_hot = np.zeros_like(y_new)
one_hot
one_hot[np.arange(len(y_new)), y_new.argmax(axis=1)] = 1
one_hot

In [ ]:
y_test_cat.shape

In [ ]:
model.evaluate(X_test_preproc, y_test_cat)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test_cat, one_hot))

In [ ]:
from sklearn.metrics import confusion_matrix

# Convertir one-hot en labels
y_test_labels = y_test_cat.argmax(axis=1)
y_pred_labels = one_hot.argmax(axis=1)

confusion_matrix(y_test_labels, y_pred_labels)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix(y_test_labels, y_pred_labels))
disp.plot()
plt.show()

In [ ]:
# # I would like to evaluate the model on each codeCPV_3 category
# dict_cpv_accuracy = {}

# for code in df['codeCPV_3'].unique():
#     print(f"Evaluating for codeCPV_3: {code}")
#     X_code = X_test[X_test['codeCPV_3'] == code]
#     y_code = y_test[y_test.index.isin(X_code.index)]

#     if not X_code.empty:
#         X_code_preproc = pipeline.transform(X_code)
#         y_code_cat = to_categorical(y_code, num_classes=8)

#         dict_cpv_accuracy['codeCPV_3'] = code
#         dict_cpv_accuracy['accuracy'] = model.evaluate(X_code_preproc, y_code_cat)[1]

#         # I would like to save the model.evaluate results for each codeCPV_3
#     else:
#         print(f"No samples found for codeCPV_3: {code}")

# cpv_accuracy_df = pd.DataFrame.from_dict(dict_cpv_accuracy, orient='index', columns=['accuracy'])
# cpv_accuracy_df.head()
from keras.utils import to_categorical

# Évaluation par codeCPV_3
results = []

for code in df['codeCPV_3'].unique():
    print(f"Evaluating for codeCPV_3: {code}")
    X_code = X_test[X_test['codeCPV_3'] == code]
    y_code = y_test[y_test.index.isin(X_code.index)]

    if not X_code.empty:
        X_code_preproc = pipeline.transform(X_code)
        y_code_cat = to_categorical(y_code, num_classes=8)
        acc = model.evaluate(X_code_preproc, y_code_cat, verbose=0)[1]
        results.append({'codeCPV_3': code, 'accuracy': acc})
    else:
        print(f"No samples found for codeCPV_3: {code}")

cpv_accuracy_df = pd.DataFrame(results)
cpv_accuracy_df.head()

In [274]:
# trie par accuracy
cpv_accuracy_df.sort_values(by='accuracy', ascending=False, inplace=True)
test = cpv_accuracy_df[cpv_accuracy_df['accuracy']>0.5]

In [ ]:
cpv_accuracy_df.head()

In [ ]:
df[df['codeCPV_3'].isin(test['codeCPV_3'])]

In [175]:
# # Dimensions
# input_dim = 220
# num_classes = 10

# # Modèle MLP
# model = Sequential()

# model.add(Input(shape=(input_dim,)))

# model.add(layers.Dense(1028, activation='relu',))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.4))

# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.3))

# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.3))

# model.add(layers.Dense(num_classes, activation='softmax'))  # Sortie pour classification

# # Compilation
# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',  # ou 'categorical_crossentropy' si y est one-hot
#     metrics=['accuracy']
# )

# # Affichage du résumé du modèle
# model.summary()

In [172]:
# es = EarlyStopping(patience=5, restore_best_weights=True, min_delta=1e-3)

# history2 = model.fit(
#         X_train_preproc, y_train_cat,
#         validation_split=0.2,
#         batch_size=32,
#         epochs=100,
#         callbacks=[es],
#         verbose=1)

In [ ]:
# Récupération des données d'entraînement et validation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Création de la figure
plt.figure(figsize=(12, 5))

# Plot de l'accuracy
plt.subplot(1, 2, 1)
plt.plot(acc, label='Accuracy')
plt.plot(val_acc, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plot de la loss
plt.subplot(1, 2, 2)
plt.plot(loss, label='Loss')
plt.plot(val_loss, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Affichage
plt.tight_layout()
plt.show()

In [173]:
# # Dimensions
# input_dim = 220
# num_classes = 10

# # Modèle MLP
# model = Sequential()

# model.add(Input(shape=(input_dim,)))

# model.add(layers.Dense(1028, activation='relu',))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.5))

# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.4))

# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.3))

# model.add(layers.Dense(num_classes, activation='softmax'))  # Sortie pour classification

# # Compilation
# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',  # ou 'categorical_crossentropy' si y est one-hot
#     metrics=['accuracy']
# )

# # Affichage du résumé du modèle
# model.summary()

In [ ]:
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=5, verbose=1, min_lr=1e-6)
early_stop = EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history3 = model.fit(
    X_train_preproc, y_train_cat,
    validation_split=0.2,
    batch_size=128,
    epochs=100,
    callbacks=[early_stop, lr_scheduler],
)

In [ ]:
def build_mlp_model(input_dim=220, num_classes=10, use_dropout=True):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))

    model.add(layers.Dense(1028, activation='relu'))
    model.add(layers.BatchNormalization())
    if use_dropout:
        model.add(layers.Dropout(0.5))

    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    if use_dropout:
        model.add(layers.Dropout(0.4))

    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    if use_dropout:
        model.add(layers.Dropout(0.3))

    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',  # or 'sparse_categorical_crossentropy'
        metrics=['accuracy']
    )

    return model

In [ ]:
model = build_mlp_model(use_dropout=False)

history4 = model.fit(
    X_train_preproc, y_train_cat,
    validation_split=0.2,
    batch_size=128,
    epochs=100,
    callbacks=[early_stop, lr_scheduler],
)

In [ ]:
# # Récupération des données d'entraînement et validation
# acc = history4.history['accuracy']
# val_acc = history4.history['val_accuracy']
# loss = history4.history['loss']
# val_loss = history4.history['val_loss']

# # Création de la figure
# plt.figure(figsize=(12, 5))

# # Plot de l'accuracy
# plt.subplot(1, 2, 1)
# plt.plot(acc, label='Accuracy')
# plt.plot(val_acc, label='Val Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.legend()

# # Plot de la loss
# plt.subplot(1, 2, 2)
# plt.plot(loss, label='Loss')
# plt.plot(val_loss, label='Val Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()

# # Affichage
# plt.tight_layout()
# plt.show()

In [171]:
# def build_mlp_model(input_dim=220, num_classes=10, use_dropout=True, l2_factor=1e-4):
#     model = Sequential()
#     model.add(Input(shape=(input_dim,)))

#     # Couche 1
#     model.add(layers.Dense(
#         1028, activation='relu',
#         kernel_regularizer=regularizers.l2(l2_factor)
#     ))
#     model.add(layers.BatchNormalization())
#     if use_dropout:
#         model.add(layers.Dropout(0.5))

#     # Couche 2
#     model.add(layers.Dense(
#         512, activation='relu',
#         kernel_regularizer=regularizers.l2(l2_factor)
#     ))
#     model.add(layers.BatchNormalization())
#     if use_dropout:
#         model.add(layers.Dropout(0.4))

#     # Couche 3
#     model.add(layers.Dense(
#         256, activation='relu',
#         kernel_regularizer=regularizers.l2(l2_factor)
#     ))
#     model.add(layers.BatchNormalization())
#     if use_dropout:
#         model.add(layers.Dropout(0.3))

#     # Sortie
#     model.add(layers.Dense(
#         num_classes, activation='softmax',
#         kernel_regularizer=regularizers.l2(l2_factor)
#     ))

#     # Compilation
#     model.compile(
#         optimizer='adam',
#         loss='categorical_crossentropy',
#         metrics=['accuracy']
#     )

#     return model

In [169]:
# model = build_mlp_model(use_dropout=False)
# history5 = model.fit(
#     X_train_preproc, y_train_cat,
#     validation_split=0.2,
#     batch_size=128,
#     epochs=100,
#     callbacks=[early_stop, lr_scheduler],
# )

In [170]:
# model = build_mlp_model(use_dropout=True)
# history5 = model.fit(
#     X_train_preproc, y_train_cat,
#     validation_split=0.2,
#     batch_size=128,
#     epochs=100,
#     callbacks=[early_stop, lr_scheduler],
# )

In [168]:
# # Récupération des données d'entraînement et validation
# acc = history5.history['accuracy']
# val_acc = history5.history['val_accuracy']
# loss = history5.history['loss']
# val_loss = history5.history['val_loss']

# # Création de la figure
# plt.figure(figsize=(12, 5))

# # Plot de l'accuracy
# plt.subplot(1, 2, 1)
# plt.plot(acc, label='Accuracy')
# plt.plot(val_acc, label='Val Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.legend()

# # Plot de la loss
# plt.subplot(1, 2, 2)
# plt.plot(loss, label='Loss')
# plt.plot(val_loss, label='Val Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()

# # Affichage
# plt.tight_layout()
# plt.show()

In [167]:
# model = build_mlp_model(use_dropout=False)
# history6 = model.fit(
#     X_train_preproc, y_train_cat,
#     validation_split=0.2,
#     batch_size=128,
#     epochs=100,
#     callbacks=[early_stop, lr_scheduler],
# )

In [166]:
# # Récupération des données d'entraînement et validation
# acc = history6.history['accuracy']
# val_acc = history6.history['val_accuracy']
# loss = history6.history['loss']
# val_loss = history6.history['val_loss']

# # Création de la figure
# plt.figure(figsize=(12, 5))

# # Plot de l'accuracy
# plt.subplot(1, 2, 1)
# plt.plot(acc, label='Accuracy')
# plt.plot(val_acc, label='Val Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.legend()

# # Plot de la loss
# plt.subplot(1, 2, 2)
# plt.plot(loss, label='Loss')
# plt.plot(val_loss, label='Val Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()

# # Affichage
# plt.tight_layout()

# plt.show()

In [165]:
# def build_mlp_model(input_dim=220, num_classes=9, use_dropout=True, l2_factor=1e-4):
#     model = Sequential()
#     model.add(Input(shape=(input_dim,)))

#     # Couche 1
#     model.add(layers.Dense(
#         1024, activation='relu',
#         kernel_regularizer=regularizers.l2(l2_factor)
#     ))
#     model.add(layers.BatchNormalization())
#     if use_dropout:
#         model.add(layers.Dropout(0.4))

#     # Couche 2
#     model.add(layers.Dense(
#         512, activation='relu',
#         kernel_regularizer=regularizers.l2(l2_factor)
#     ))
#     model.add(layers.BatchNormalization())
#     if use_dropout:
#         model.add(layers.Dropout(0.3))

#     # Couche 3
#     model.add(layers.Dense(
#         256, activation='relu',
#         kernel_regularizer=regularizers.l2(l2_factor)
#     ))
#     model.add(layers.BatchNormalization())
#     if use_dropout:
#         model.add(layers.Dropout(0.2))
#     #couche 4
#     model.add(layers.Dense(
#         128, activation='relu',
#         kernel_regularizer=regularizers.l2(l2_factor)
#     ))
#     model.add(layers.BatchNormalization())
#     if use_dropout:
#         model.add(layers.Dropout(0.1))

#     # Sortie
#     model.add(layers.Dense(
#         num_classes, activation='softmax',
#         kernel_regularizer=regularizers.l2(l2_factor)
#     ))

#     # Compilation
#     model.compile(
#         optimizer='adam',
#         loss='categorical_crossentropy',
#         metrics=['accuracy']
#     )

#     return model

In [164]:
# model = build_mlp_model(use_dropout=False)
# history7 = model.fit(
#     X_train_preproc, y_train_cat,
#     validation_split=0.2,
#     batch_size=128,
#     epochs=100,
#     callbacks=[early_stop, lr_scheduler],
# )

In [163]:
# # Récupération des données d'entraînement et validation
# acc = history7.history['accuracy']
# val_acc = history7.history['val_accuracy']
# loss = history7.history['loss']
# val_loss = history7.history['val_loss']

# # Création de la figure
# plt.figure(figsize=(12, 5))

# # Plot de l'accuracy
# plt.subplot(1, 2, 1)
# plt.plot(acc, label='Accuracy')
# plt.plot(val_acc, label='Val Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.legend()

# # Plot de la loss
# plt.subplot(1, 2, 2)
# plt.plot(loss, label='Loss')
# plt.plot(val_loss, label='Val Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()

# # Affichage
# plt.tight_layout()
# plt.show()